In [56]:
import csv
import nltk
import pandas as pd
import re

from tqdm import tqdm

In [57]:
df = pd.read_csv('/Users/osama/Documents/finalNlu/data/dataSplitting/dataset_with_length_of_fields.csv')

In [58]:
df.head(1)

,Unnamed: 0,id,title,text,verdict,comment1,comment2,score,titleAndText,titleAndTextLength,textLength,comment1Length,comment2Length
0,0,4pcrle,aita for wanting to celebrate my birthday,i turned 30 on monday. i don't have a problem ...,nta,"i think you know your nta, it seems like you j...",definitely nta. i suffer from depression and a...,15,aita for wanting to celebrate my birthday i tu...,475,468,194,42


In [59]:
withTitleDf = df.copy()
withTitleDf = withTitleDf.drop(['title', 'text', 'textLength'], axis=1)
withTitleDf.head(1)

,Unnamed: 0,id,verdict,comment1,comment2,score,titleAndText,titleAndTextLength,comment1Length,comment2Length
0,0,4pcrle,nta,"i think you know your nta, it seems like you j...",definitely nta. i suffer from depression and a...,15,aita for wanting to celebrate my birthday i tu...,475,194,42


In [60]:
len(withTitleDf)

270709

In [61]:
withoutTitleDf = df.copy()
withoutTitleDf = withoutTitleDf.drop(['title', 'titleAndText', 'titleAndTextLength'], axis=1)
withoutTitleDf.head(1)

,Unnamed: 0,id,text,verdict,comment1,comment2,score,textLength,comment1Length,comment2Length
0,0,4pcrle,i turned 30 on monday. i don't have a problem ...,nta,"i think you know your nta, it seems like you j...",definitely nta. i suffer from depression and a...,15,468,194,42


In [62]:
len(withoutTitleDf)

270709

In [63]:
withTitleDfLessThan512 = withTitleDf[withTitleDf['titleAndTextLength'] < 512] 
withTitleDfLessThan1024 = withTitleDf[withTitleDf['titleAndTextLength'] < 1024] 

In [64]:
len(withTitleDfLessThan512)

181283

In [65]:
len(withTitleDfLessThan1024)

269163

In [66]:
withoutTitleDfLessThan512 = withoutTitleDf[withoutTitleDf['textLength'] < 512] 
withoutTitleDfLessThan1024 = withoutTitleDf[withoutTitleDf['textLength'] < 1024] 

In [67]:
len(withoutTitleDfLessThan512)

186567

In [68]:
len(withoutTitleDfLessThan1024)

269260

In [69]:
withTitleDfLessThan512singleComment = withTitleDfLessThan512.drop(['comment2', 'titleAndTextLength', 'comment1Length', 'comment2Length'], axis=1).copy().rename(columns={"comment1": "comment", "titleAndText": "text"})
withTitleDfLessThan1024singleComment = withTitleDfLessThan1024.drop(['comment2', 'titleAndTextLength', 'comment1Length', 'comment2Length'], axis=1).copy().rename(columns={"comment1": "comment", "titleAndText": "text"})
withoutTitleDfLessThan512singleComment = withoutTitleDfLessThan512.drop(['comment2', 'textLength', 'comment1Length', 'comment2Length'], axis=1).copy().rename(columns={"comment1": "comment"})
withoutTitleDfLessThan1024singleComment = withoutTitleDfLessThan1024.drop(['comment2', 'textLength', 'comment1Length', 'comment2Length'], axis=1).copy().rename(columns={"comment1": "comment"})

In [70]:
withTitleDfLessThan512singleComment.head(1)

,Unnamed: 0,id,verdict,comment,score,text
0,0,4pcrle,nta,"i think you know your nta, it seems like you j...",15,aita for wanting to celebrate my birthday i tu...


In [71]:
withTitleDfLessThan1024singleComment.head(1)

,Unnamed: 0,id,verdict,comment,score,text
0,0,4pcrle,nta,"i think you know your nta, it seems like you j...",15,aita for wanting to celebrate my birthday i tu...


In [72]:
withoutTitleDfLessThan512singleComment.head(1)

,Unnamed: 0,id,text,verdict,comment,score
0,0,4pcrle,i turned 30 on monday. i don't have a problem ...,nta,"i think you know your nta, it seems like you j...",15


In [73]:
withoutTitleDfLessThan1024singleComment.head(1)

,Unnamed: 0,id,text,verdict,comment,score
0,0,4pcrle,i turned 30 on monday. i don't have a problem ...,nta,"i think you know your nta, it seems like you j...",15


In [74]:
withTitleDfLessThan512doubleComment = dict(comment=withTitleDfLessThan512.filter(regex="^comment[1|2]$").columns)
withTitleDfLessThan512doubleComment = pd.lreshape(withTitleDfLessThan512, withTitleDfLessThan512doubleComment)
withTitleDfLessThan512doubleComment= withTitleDfLessThan512doubleComment.drop(['titleAndTextLength', 'comment1Length', 'comment2Length'], axis=1).copy().rename(columns={"titleAndText": "text"})

withTitleDfLessThan1024doubleComment = dict(comment=withTitleDfLessThan1024.filter(regex="^comment[1|2]$").columns)
withTitleDfLessThan1024doubleComment = pd.lreshape(withTitleDfLessThan1024, withTitleDfLessThan1024doubleComment)
withTitleDfLessThan1024doubleComment= withTitleDfLessThan1024doubleComment.drop(['titleAndTextLength', 'comment1Length', 'comment2Length'], axis=1).copy().rename(columns={"titleAndText": "text"})

In [75]:
withoutTitleDfLessThan512doubleComment = dict(comment=withoutTitleDfLessThan512.filter(regex="^comment[1|2]$").columns)
withoutTitleDfLessThan512doubleComment = pd.lreshape(withoutTitleDfLessThan512, withoutTitleDfLessThan512doubleComment)
withoutTitleDfLessThan512doubleComment= withoutTitleDfLessThan512doubleComment.drop(['textLength', 'comment1Length', 'comment2Length'], axis=1).copy()

withoutTitleDfLessThan1024doubleComment = dict(comment=withoutTitleDfLessThan1024.filter(regex="^comment[1|2]$").columns)
withoutTitleDfLessThan1024doubleComment = pd.lreshape(withoutTitleDfLessThan1024, withoutTitleDfLessThan1024doubleComment)
withoutTitleDfLessThan1024doubleComment= withoutTitleDfLessThan1024doubleComment.drop(['textLength', 'comment1Length', 'comment2Length'], axis=1).copy()

In [76]:
withTitleDfLessThan512singleComment.to_csv("subset_with_title_less_than_512_single_comment.csv")  
withTitleDfLessThan1024singleComment.to_csv("subset_with_title_less_than_1024_single_comment.csv")  
withoutTitleDfLessThan512singleComment.to_csv("subset_without_title_less_than_512_single_comment.csv")  
withoutTitleDfLessThan1024singleComment.to_csv("subset_without_title_less_than_1024_single_comment.csv") 
withTitleDfLessThan512doubleComment.to_csv("subset_with_title_less_than_512_double_comment.csv")  
withTitleDfLessThan1024doubleComment.to_csv("subset_with_title_less_than_1024_double_comment.csv")  
withoutTitleDfLessThan512doubleComment.to_csv("subset_without_title_less_than_512_double_comment.csv")  
withoutTitleDfLessThan1024doubleComment.to_csv("subset_without_title_less_than_1024_double_comment.csv")  

In [77]:
print(len(withTitleDfLessThan512singleComment))
print(len(withTitleDfLessThan1024singleComment))
print(len(withoutTitleDfLessThan512singleComment))
print(len(withoutTitleDfLessThan1024singleComment))
print(len(withTitleDfLessThan512doubleComment))
print(len(withTitleDfLessThan1024doubleComment))
print(len(withoutTitleDfLessThan512doubleComment))
print(len(withoutTitleDfLessThan1024doubleComment))

181283
269163
186567
269260
362566
538326
373134
538520
